In [2]:
# libraries
import tensorflow as tf
from tensorflow.keras.layers import Input, Dropout, Conv2D, Dense, Flatten
from tensorflow.keras.models import Model, Sequential

models = {}

In [3]:
def dense_model(input_shape, output_shape, name='dense_model', **kwargs):
    # get hyper-parameters
    dropout = kwargs.get('dropout', 0.4)
    dense_units = kwargs.get('dense_units', 64)
    hidden_activation = kwargs.get('hidden_activation', 'relu')
    layers_count = max(kwargs.get('layers_count', 2), 1)
    
    # hidden network
    hidden = Sequential()
    ### first layer
    hidden.add(Dense(dense_units, 
                     activation=hidden_activation, 
                     input_shape=input_shape))
    hidden.add(Dropout(rate=dropout))
    ### append others
    for _ in range(layers_count - 1):
        hidden.add(Dense(dense_units, 
                         activation=hidden_activation))
        hidden.add(Dropout(rate=dropout))
    ### final layers      
    hidden.add(Flatten())
    hidden.add(Dense(output_shape, activation='softmax'))    
    
    # input and output
    tf_input = Input(shape=input_shape)
    tf_output = hidden(tf_input)
    
    # create model
    model = Model(tf_input, tf_output)
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

models['dense_model'] = dense_model

In [4]:
def get_model(input_shape, output_shape, model_id, 
              model_name="sudoku_net", **kwargs):
    return models[model_id](input_shape, output_shape, model_name, kwargs)